<br> <br> <br>

<center style="font-size: 40px">
<strong>Homework 1</strong>
</center>

<br> <br>

In this homework we look at the data set `cars-raw.csv`, which consists of used Toyota Corollas rectnly posted on <https://www.cars.com/>. The data were webscraped from this website using the code given below.

You can work on the homework using the provided csv file posted on Canvas or else, you can webscrape the data for the make and model of your choice within certain distance from some other zip code in the Phoenix area. In the latter case, you need to make sure you have at least 250 cars webscraped. In addition, must contain at least a couple of cars with "Not Priced" listed under the price.


<br>

### Webscraping &nbsp; (Optional)

<br>

If you decide to webscrape, you should install the modules `requests` and `beautifulsoup4`. They are used for webscraping and parsing web pages.

Here is the code that was used to produce `cars-raw.csv` file posted on Canvas.

In [1]:
# pip install requests
# pip install beautifulsoup4
# pip install matplotlib
# pip install pandas
# pip install sklearn

In [2]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#need this later
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

Go to <https://cars.com> and choose the following search conditions (or similar conditions as instructed above):

<br>

<div>
<img src="cars.com.png" width="600"/>
</div>

<br> <br>

By default, only 20 cars are listed per page and at the bottom of the page you can go to next page.

Here are the web addresses for the search conditions used as in the image.

url1 = "https://www.cars.com/shopping/results/?stock_type=used&makes%5B%5D=toyota&models%5B%5D=toyota-corolla&list_price_max=&maximum_distance=200&zip=85281"

url2 = "https://www.cars.com/shopping/results/?page=2&page_size=20&list_price_max=&makes[]=toyota&maximum_distance=200&models[]=toyota-corolla&stock_type=used&zip=85281"

url3 = "https://www.cars.com/shopping/results/?page=3&page_size=20&list_price_max=&makes[]=toyota&maximum_distance=200&models[]=toyota-corolla&stock_type=used&zip=85281"

You can see that there is a slight difference between the addresses of the 2nd, 3rd, 4th, and other pages. The difference is in the part `page=2`, `page=3`, `page=4`, etc., in the address. 

The first page has different address from the rest, but if you replace `page=1` in the `url2`, you will get to the same page as the above `url1`. So, we will use this version. (Of course, if you want to webscrape on your own, the urls will be slightly different and you would need to make appropriate adjustments).


In [ ]:
# Set the url
# cars.com, used toyota corolla, no max price, within 200 mi from the zip code 85282

prepage = "https://www.cars.com/shopping/results/?"
postpage = "&page_size=20&list_price_max=&makes[]=toyota&maximum_distance=200&models[]=toyota-corolla&stock_type=used&zip=85282"

url_list = []

npages = 15  #we webscrape first 15 pages (you may need to adjust this, if your search
             #has less pages)

## create a list of url's for the desired web pages
for i in range(npages):
    new_url = prepage + 'page=' + str(i+1) + postpage
    url_list.append(new_url)



Next, we create lists of tags for the three html tags that contain the data about the year the car is made, mileage and price. These tags were identified after inspecting the first web page (right-hand click on the webpage, then click on Inspect). If you don't know html, you don't have to try and don't worry, it was already done for you.

Here are the python lists of the three tag types that we are going to create:

- `vehicles_tags` - is a python list of `h2` tags of class name `title` <br> (For example, for the first car, the content of this tag is `2019 Toyota Corolla LE`)
<br>

- `mileages_tags` - is a python list of `div` tags of class name `mileage` <br> (For example, for the first car, the content of this tag is `47,925 mi.`) 
<br>

- `prices_tags` - is a python list of `span` tags of class name `primary-price` <br> (For example, for the first car, the content of this tage is `$17,459`)

<br>

To create these python lists, we make a `for`-loop running over the pages and in each iteration we do the following:

- webscrape data from the current page using `requests` module

- use `BeautifulSoup` to turn the obtained `requests` object into a `soup` object to parse the content

- find all tags of the desired type and class name, using `soup.find_all()` method

- we create python lists `vehicles`, `mileages` and `prices`, which consist of the contents of the corresponding tags.



In [ ]:
vehicles = []
mileages = []
prices = []

for i in range(npages):
    print('working on page'+str(i+1))
    page = requests.get(url_list[i])  #get a current html page 
    soup = BeautifulSoup(page.content, 'html.parser') #turn response into a soup object to parse html content
    
    vehicles_tags = soup.find_all('h2', attrs={'class': 'title'})  #extract h2 tags of class 'title' (happen to contain year)
    mileages_tags = soup.find_all('div', attrs={'class': 'mileage'}) #extract div tags of class 'mileage'
    prices_tags = soup.find_all('span', attrs={'class', 'primary-price'}) #extract span tags of class 'primary-price'
    
    for j in range(len(vehicles_tags)):
        vehicles.append(vehicles_tags[j].get_text()) #append the vehicles list with current vehicle titles
        mileages.append(mileages_tags[j].get_text()) #append the mileages list with current mileages
        prices.append(prices_tags[j].get_text()) #append the prices list with current prices
    



Next, we put the three lists into the `pandas` data frame `rawdf`.
<br>

In [ ]:
rawdf = pd.DataFrame({'vehicle': vehicles, 'mileage': mileages, 'price': prices})

rawdf.head(6)

We then create a csv file `cars-raw.csv` from the data frame `rawdf`, usding `to_csv()` method.

In [ ]:
##store the data into the csv file cars-raw.csv
# rawdf.to_csv("cars-raw.csv", header=True, index=False)

<br>
<p style="font-size:16px">
This ends the optional part of the homework. From now on, you either use the provided file `cars-raw.csv` from Canvas, or your own version created using the above code. To use your file, you must make sure that your searching conditions are such that you have over 250 observations, i.e. cars.
</p>

<br>

### Data Wrangling

**(1)**

$\bullet$ Read in the data from `cars-raw.csv`, the file that either you took from Canvas, or created by yourself following the code above. Store the data into the `pandas` data frame object called `rawdf.`

To do that, use `pandas` method `read_csv`. If necessary, see the documentation for this method. You can set the `header` argument to `header=0`, although it's not necessary for this file.

$\bullet$ Then, print out the first 5 rows, using the `head` method.

You should get something like this:
    
<img src="rawdf.png" style="width: 300px;" align="left">

In [ ]:
## your code here (for reading in the file and showing the head of the data frame)

rawdf = pd.read_csv('mycars-raw.csv')
rawdf.head(5)



<br>

In the following code cells, we need to do some **data wrangling** (which is a fancy name for transforming data into a useable format).

We cannot work with this data frame: we need `mileage` and `price` to be numeric values (not strings with commas, mi., and $). Also, we want to extract the year from `vehicle` strings.

First, we clean the price column by removing $ and the comma.

In [ ]:
##df will be a new, transformed data frame
df = rawdf

##removing $ symbol in the price column
##in regular expression manipulations, $ has a special role. So, to tell python we mean $ literally, we use \$ instead 
df['price'] = df['price'].replace({'\$': '', ',': ''}, regex=True)

##creating the column 'year' by taking first 4 characters of the `vehicle` column
year = [int(df['vehicle'][i][:4]) for i in range(len(df))]

##re-defining df data frame by including the year list
df = pd.DataFrame({'vehicle': df['vehicle'], 'year': year, 'mileage': df['mileage'], 'price': df['price']})

df

**(2)**

Now, it's your turn. After each of the following steps plot first couple of rows to see whether everything seems okay.

$\bullet$ Remove `mi.` and `,` in the mileage column. You can mimic the code from above. Also, print the first couple of rows of the new version of the dataframe `df` using  the `head()` method.

$\bullet$ Next, create a list `year`, so that its $i$-th entry is the year the $i$-th car was manufactured. To do that, for each $i$, just extract first four characters of the corresponding `vehicle` entry and put it into the `year` list.

$\bullet$ Then, re-define `df` to be the data frame with columns `vehicle`, `year`, `mileage` and `price`, in that order. In other words, you just need to insert the list `year` into `df` as its 2nd column. You can consult StackOverflow or `pandas` documentation to see how to bind lists/series into a data frame.

$\bullet$ Then, use `head()` method to see whether everything looks as it should.



In [ ]:
## your code here
##remove "mi." and "," in the mileage column
df['Mileage'] = df['Mileage'].replace({',': '', 'mi.': ''}, regex=True)

##creating the column 'year' by taking first 4 characters of the `vehicle` column
df['Year'] = [int(df['Car'][i][:4]) for i in range(len(df))]

##re-defining df data frame by including the year list
df = pd.DataFrame({'Car': df['Car'], 'Year': df['Year'], 'Mileage': df['Mileage'], 'Price': df['Price']})


<br>

**(3)**

Unfortunately, our csv file (and thus, the data frame `df`) contains some cars whose price is not listed. 
 
$\bullet$ Create the data frame `notpriceddf` which consists precisely of those cars (i.e. rows) in `df` whose `price` entry has value `Not Priced`.

$\bullet$ In addition, create data frame `cleandf`, which consists precisely of those cars in `df` which do have price, i.e whose `price` entry is NOT "Not Priced".

$\bullet$ Print out the number of observations in the data frames `notpriceddf` and `cleandf`. You can use the method `shape()` or function `len()`. How many non-priced cars are there in the original data set?

You will soon estimate a reasonable (i.e. expected) price of the cars with missing price. By the way, cleaning data by removing useless or missing data is also 
called **data munging**.



In [ ]:
##your code here
nopricedf = df[df.Price.str.contains('Not')]

cleandf = df[~df.Price.str.contains('Not')]

print(len(df), len(nopricedf), len(cleandf))



<br>

**(4)**

This is all nice, but the columns `mileage` and `price` (and maybe also `year`, depending how you created it) still have strings as entries, i.e. are not numeric. We need them to be numeric.

$\bullet$ Use the method `apply()` and the function `pd.to_numeric` to convert the entries of these three columns into numeric values:

`cleandf[<list of columns>] = cleandf[<list of columns>].apply(pd.to_numeric)`


where `<list of columns>` should be replaced by an appropriate list. 

$\bullet$ Print the first couple of rows to see whether everything is okay.



In [ ]:
##your code here
cleandf[['Year', 'Mileage', 'Price']] = cleandf[['Year', 'Mileage', 'Price']].apply(pd.to_numeric)


<br>

$\bullet$  To check that the types of the data are correct (i.e. numeric, which in our case should be integers), print out the list of types of the values in the 1st row. To do that, use the for loop within the brackets, as well as the `iloc` method to call all the column indexes within the for-loop.

In [ ]:
##your code here
[print(type(cleandf.iloc[0, i])) for i in range(0,4)]


<br>

<p style="font-size:16px">
Now we are done with data wrangling and munging. <br> <br>
Our next step is to create regression models.
</p>

<br>

<br>

**(5)**

$\bullet$ Create variable `X`, which is a numpy array (ndarray) that consists of `year` and `mileage` columns of `cleandf`. Use `pandas` method `to_numpy()` to do that.

$\bullet$ In addition, create a numpy array `y`, which is a vector of entries of the `price` column. Use `pandas` data frame attribute `values`.

$\bullet$ Print out first 5 observations from both `X` and `y`.

Also, print out the dimensions (i.e. shapes) of `X` and `y`. Also, p

In [ ]:
## your code here
x = cleandf[['Year', 'Mileage']].to_numpy()
y = cleandf['Price'].values

print(x[0:5], '\n', y[0:5].reshape(-1,1), '\n', x.shape, '\n', y.shape)



<br>  
  
**(6)**

$\bullet$ Now let's plot `price` vs. `mileage` (so, `price` on the $y$-axis, and `mileage` on the $x$-axis). `matplolib`'s `scatter` method. Set the `s` argument (for size of the points in the scatter plot) to be `s=10`. Include appropriate x and y labels, and the title of the plot.


In [ ]:
##your code here
plt.scatter(x[:,1], y, s=10)
plt.title('Price Based on Mileage')
plt.xlabel('Mileage')
plt.ylabel('Price')


<br>

**(7)**

$\bullet$ Create a `sklearn` model `mod0` that corresponds to the regression model of `price` onto mileage:

$$\begin{align}\text{price} &= \beta_0 + \beta_1\cdot \text{mileage} + \varepsilon \nonumber \\ 
\nonumber \\ 
\widehat{E[\text{price}|\text{mileage}]}&= \hat{\beta}_0 + \hat{\beta}_1 \cdot \text{mileage}\end{align}$$

<br>

To do that, first create a variable `x` which is the column of `X` with mileage data. To make the model, use `LinearRegression()` method from `sklearn`, but first you would need to set

`x.shape = (len(x),1)`

This is because `LinearRegression()` method expects `x` to have two dimensions.

<br>

$\bullet$ Print out python tuple of coefficients $(\hat{\beta}_0, \hat{\beta}_1)$

<br>

In [ ]:
## To apply LinearRegression() function, we need to set  `x.shape` to `(506,1)`
x = X[:,1]
x.shape = (len(x),1)

## your code here
mod0 = LinearRegression().fit(x,y)
b = (mod0.intercept_, mod0.coef_)
print(b)



<br>

**(8)**

$\bullet$ Using the`sklearn` method `predict()`, create a variable `yhat`, which is a vector of predicted values (i.e. the estimates $\displaystyle \widehat{E[\text{price}|\text{mileage}]}$) for all mileage values from the array `x`. 


$\bullet$ Plot the scatter plot of `price` vs. `mileage` as above, but now include the regression line.

In [ ]:
##your code here



<br>

$\bullet$ Plot the scatter plot of `yhat` vs. `y` (so, `yhat` is on $y$-axis and `y` is on $x$-axis). Also, include the identity line `yhat=y`.

<br>

In [ ]:
##your code here



Looks like our prediction `yhat` tends to underastimate price when the actual price is high (which corresponds to a very low mileage). We will take a look at another model, but to compare the two models, we will modify this one.

In the following code, we split the data: 75% of the data will be used for fitting/training, and 25% for testing how good the model is. 

Our `mod1` model will be just as above, i.e. $\text{price} = \beta_0 + \beta_1\cdot \text{mileage} + \varepsilon$, but this time it will be fitted/trained only on these 75% of the data

In [ ]:
X1train, X1test, y1train, y1test = train_test_split(x,y, random_state=123,test_size=.25)

mod1 = LinearRegression()
mod1.fit(X1train,y1train)

mod1.intercept_ , mod1.coef_   #print bhat_0 and bhat_1 


<br>
Note that the coefficients are similar to the model when we used 100% of the data, although not quite the same.
<br> <br>

There are various metrics used to measure how good a model is. We will use Root Mean Squared Error:

$$RMSE = \sqrt{{1\over n}\sum_{i=1}^n(\hat{y}_i-y_i)^2}$$

where $n$ is the sample size. 

Recall that in statistics, the sum $\displaystyle \sum_{i=1}^n(\hat{y}_i-y_i)^2$ is called **sum of squares of errors**,  or sum of squares of residuals, (denoted by SSE).

<br>

We use RMSE both to see how well the model fits to the train data, and to see how well it predicts the price for the $X$-data coming out of the trained sample.
<br>

In [ ]:
#find predictions of price for X-values in the trained data
y1train_hat = mod1.predict(X1train)

#find predictions of price for X-values in the test data
y1test_hat = mod1.predict(X1test)

In [ ]:
##computing root mean squared errors

rmse1_train = np.sqrt(np.mean(np.power(y1train_hat-y1train, 2)))

print('rmse1_train:', rmse1_train)

rmse1_test = np.sqrt(np.mean(np.power(y1test_hat-y1test, 2)))

print('rmse1_test: ', rmse1_test)

<br>

**(9)**

We are now going to fit the model:

$$\begin{align}\text{price} &= \beta_0 + \beta_1\cdot \text{age} +  \hat{\beta}_2\cdot \text{mileage} + \hat{\beta}_3\cdot \text{mileage}^2 + \varepsilon \nonumber \\ 
\nonumber \\
\widehat{E[\text{price}|\text{mileage}]}&= \hat{\beta}_0 + \hat{\beta}_1 \cdot \text{age} + \hat{\beta}_2\cdot \text{mileage} + \hat{\beta}_3\cdot \text{mileage}^2\end{align}$$

$\bullet$ Create a numpy ndarray `X2` whose columns are made of `age`, `mileage` and `mileaage`$^2$, where `age` is the age of the car.

To get age, you could just subtract the year of the today's date from the first column of `X`. But, you can make your code flexible by running the followng cell, which creates variable `current_year` from the today's date. This way, when you run this notebook next year and the year after..., you wouldn't need to change the code. However, you don't need to use `current_year` if you don't want.

To create `X2`, you may want to use `numpy` method `column_stack()`. You can read the documentation online. 

<br>

$\bullet$ Print out first 5 rows of the ndarray `X2` to see whether everything is okay.


$\bullet$ Plot the scatter plot of `price` vs. `age`.

<br>


In [ ]:
from datetime import date
current_year = date.today().year

age = current_year - X[:,0]


In [ ]:
age = current_year - X[:,0]

milsq = np.power(X[:,1],2)

##your code here (creating X2)




In [ ]:
##your plot code here




<br>

**(10)**

Now it's your turn to fit this model

$$\widehat{E[\text{price}|\text{mileage}]}= \hat{\beta}_0 + \hat{\beta}_1 \cdot \text{age} + \hat{\beta}_2\cdot \text{mileage} + \hat{\beta}_3\cdot \text{mileage}^2$$

by repeating the steps we have done above when fitting `mod1`. All you need to do is just to change the names of some variables.

$\bullet$ Split the data: create `X2train`, `X2test`, `y2train`, `y2test` by running `train_test_split()` with arguments `X2` and `y` (rather than `x` and `y` in the first model). Use the same `random_state=123`, to get the same split as in `mod1`. Then, create an appropriate model `mod2` with columns of `X2train` as predictors. Fit the model and show the intercept and other coefficients.

$\bullet$ Create prediction vectors `y2train_hat` and `y2test_hat` based on the correpsonding predictor values `X2train` and `X2test`, respectively.

$\bullet$ Compute root mean squared errors for both train and test $y$-values.

$\bullet$ Plot the scatter plot of `y2train_hat` `y2train`.


In [ ]:
## your code here; create X2train, X2test, y2train, y2test 
X2train, X2test, y2train, y2test = train_test_split(X2,y, random_state=123,test_size=.25)


##create mod2, fit it, and print (bhat_0, bhat_1)



In [ ]:
##your code here; create y2train_hat and y2testhat




In [ ]:
##your code here; compute root mean squared errors for train and test data




In [ ]:
## your code here; plot y2train_hat vs. y2train




Run the following code to print the difference between root mean squared errors of the two models, in both the train sample and in the test sample.

In [ ]:
print('[rmse1_train, rmse2_train]:', np.round([rmse1_train, rmse2_train],2))
print('[rmse1_test, rmse2_test]: ', np.round([rmse1_test, rmse2_test], 2))

<br>

You should have gotten that `rmse1_train` > `rmse2_train`, as well as `rmse1_test` > `rmse2_test`.

The fact that `rmse1_train` > `rmse2_train` suggests that the model 2 better fits the train data. This is not surprising, since `mod1` is a *reduced* version of `mod2`, i.e. is its submodel. 

However, in general, if one model has smaller RMSE for the train data, that still doesn't mean it predicts better (as we shall see when we talk about bias-variance tradeoff). The true test about which model is better is to look at the test data, for predictions out of sample. Nevertheless, in our case `mod2` beats `mod1` there, as well. That is, the RMSE of `mod2` is smaller, suggesting that `mod2` is a better fit. 

**Note**: It is possible that for some other split of the data into 75-25 ratio, `mod1` happens to have smaller RMSE than `mod2`. In order for one model to be superior over another, we need that *ON AVERAGE* (i.e. on average from test sample to test sample), the RMSE of the first model be smaller than RMSE of the second.